In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#@author: Kelle Clark, Andrew Florian, Xinyu Xiong
#Created on Tue Feb  4 10:05:49 2020
#CSCI 6040 Project 1 Text Generation
#PHASE 1: READING IN THE CORPUS

#Various folders of .txt files were created in the CSCI6040 Team Project 1 folder
#to be used for testing our application during develpment
#/Short Test Data
# has 3 .txt files each about 4KB
#/Med test Data 
# has 2 .txt files one of 119KB (Tragedy of Macbeth) and 6.5MB (big)
#/Grande test Data (the 18-document-gutenburg-copus but with 19? files cleaned using the 
#boilerplate.ipynb -author Andrew Florian and resulting files 
#shared on Canvas in Project 1 discussion forum)
# has 19 .txt files with a total of 11.8MB

In [205]:
#we needed the help of a few packages...import all those at once
import langid
import itertools 
import nltk
import numpy
import os
import pandas
import random
import re
import string
import sys
from collections import Counter
from math import log10
from matplotlib.pyplot import yscale, xscale, title, plot
from nltk.tokenize import word_tokenize, sent_tokenize

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

#from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM
#from keras.utils import np_utils
#from keras.callbacks import ModelCheckpoint


In [206]:
#this method will read in only the .txt files in the folderpath provided as input
#an error will be returned if unable to read a file
#an optional extension of the code is to determine the language of the text so that
#we could generte text by language choice.

def textByFile(folderpath):
    textfiles = [f for f in os.listdir(folderpath) if '.txt' in f]
    filedictionary ={}
    
    for f in textfiles:
        try:
            substring = ' '
            with open(folderpath + '/'+ f) as filetext:
                #print(f"the language of file "+f+" is {nltk.language(filetext)}")
                substring = substring.join(line.strip() for line in filetext)
                filetext.close()
                filedictionary[f] = substring
        except:
            print("Error reading file:" + f)
            
    return filedictionary

In [207]:
#method to print out the read in file based on length of file...for files with more than 500 words
#the method only prints out the first 50 and last 50 words
#an execption is printed if the corpus does not exist

def printoutfiles(indictionary,folderpath):
    tempcounter = 0
    for fileidentifier in indictionary:
        instring = indictionary[fileidentifier]
        tempcounter = tempcounter + len(instring)
        try:
            print(f"The {fileidentifier} file in {pathname} has {len(instring)} tokens. \n")
            if len(instring) > 500: 
                 print(f"The first & last 50 tokens of this file are:\n" + instring[:50] + " . . . "+ instring[-50:]+"\n")
            else:
                 print(f"The tokens in file {fileidentifier} are: \n {instring} \n")
        except:
            print("The string from file {f} does not exist.\n")
    print(f"The collection of all .txt files in {pathname} has {tempcounter} tokens. \n")
    return

In [208]:
#the pathname for the folder containing the files to be read in is currently hardcoded
#***if you want to change the pathname, uncomment the 2nd line and put in your pathname of choice
#***if you want to use the test data, leave the hardcoded pathname (do not uncommnet 2nd line) 
#***there are several test data sets in our folder

pathname = 'Test Data/short test data'
#pathname = 'your choice of path here'
filedict = textByFile(pathname)

#create a fileID dictionary and use fileId for files instead of filenames
fileIDdict = {}

i = 0
for f in filedict:
    fileIDdict[i] = filedict[f] 
    i = i + 1

In [102]:
#printing out the the files from the folder either by file name or fileID
printoutfiles(filedict,pathname)
#printoutfiles(fileIDdict,pathname)

The Testset1.txt file in Test Data/short test data has 62 tokens. 

The tokens in file Testset1.txt are: 
 The cat named Bob is DAMN good! He is the best C-A-T on Earth. 

The Testset2.txt file in Test Data/short test data has 296 tokens. 

The tokens in file Testset2.txt are: 
 What do we do if there are hyphens, capitols in the middle of a word, misspellings? Do we want to keep track of words at the beginning of sentences?  Our test data need to have enough words that have high frequency.  What do we do with words like 12.exe. or http://www.weirdo.com and #9 and 1999? 

The Testset3.txt file in Test Data/short test data has 113 tokens. 

The tokens in file Testset3.txt are: 
 This damn cat whose name is Bob is no good. My dog is better. Cats should not exist on Earth.  Dogs are the best. 

The corpus of all .txt files in Test Data/short test data has 471 tokens. 



In [209]:
# normalize the tokens by casefolding and removing punctuation to standardize it
# this code only keeps regexpressions [A-Za-z]

def tokenize_wordListByFile(indict):
    words = []
    sumofwords = 0
    #creating a regular expression for matching
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    fileid = 0
    for i in range(len(indict)):
        tokens = word_tokenize(indict[i])
        tokens = [w.lower() for w in tokens]
        
        #remove punctuation from words, only include words that are A-Za-z
        punct_free_words = [re_punc.sub('', w) for w in tokens if w.isalpha()]
        tempstring = " "
        for word in punct_free_words:          
            tempstring = tempstring + word + " "
        words.append(tempstring)
        sumofwords = sumofwords + len(words[i])
        
    print(f"The word_tokenize from NLTK identified tokenized {sumofwords} words using A-Za-z.")
    return words

In [210]:
#clean up corpus by file and split string between expressions involving A-Za-z
#send in the dictionary using the fileIDs
tokenfilelist = tokenize_wordListByFile(fileIDdict)
print(tokenfilelist)

The word_tokenize from NLTK identified tokenized 423 words using A-Za-z.
[' the cat named bob is damn good he is the best on earth ', ' what do we do if there are hyphens capitols in the middle of a word misspellings do we want to keep track of words at the beginning of sentences our test data need to have enough words that have high frequency what do we do with words like or http and and ', ' this damn cat whose name is bob is no good my dog is better cats should not exist on earth dogs are the best ']


In [211]:
#we have the different files tokenized, in the variable tokenfilelist
#method below creates one corpus from the string of tokens in each file 

def createOneCorpus(inlist):
    temp = " "
    for i in range(len(inlist)):
        temp = temp + inlist[i] + " "
    return temp

def printcorpus(instring):
    if len(instring) > 500: 
        print(f"The first & last 50 tokens of this corpus are:\n  {instring[:50]} \t ... {instring[-50:]}\n")
    else:
        print(f"The tokens in the corpus are: \n {instring} \n")

tokencorpus = createOneCorpus(tokenfilelist)
printcorpus(tokencorpus)

The tokens in the corpus are: 
   the cat named bob is damn good he is the best on earth   what do we do if there are hyphens capitols in the middle of a word misspellings do we want to keep track of words at the beginning of sentences our test data need to have enough words that have high frequency what do we do with words like or http and and   this damn cat whose name is bob is no good my dog is better cats should not exist on earth dogs are the best   

